### 1 Setting Up The Environment

The first step is to run the setup file in the virtual environment set up by Alastair, as shown below. 
This is another notebook in the workspaces that loads the correct set of python libraries and packages that ensure that all scripts equally.
In cell 8 of setup make sure to change :

> ag1k_dir = '/Users/Utente/Documents/UROP2018/'

to your own folder (in which you have vector_ops). 
<br>Also change the output directory location in the cell below.

In [46]:
%run setup.ipynb
output_dir = Path('/Users/Utente/Documents/UROP2018/Outputfiles/vector/observatory/analysis/107-sfs-data-request-imperial')

### 2 Extract Outgroup Data

The outgroups that are available are : Arab, Quad, Meru, Mela, Chri, Epir.
<br> Only one sequence is available for Chri and Epir
<br> Arab, Quad, Meru, Mela, contain the sequences of several individuals. 
<br> Arab has 12 individuals sequences, Quad and Meru have 10 individuals each and Mela has 4.


In [20]:
@functools.lru_cache(maxsize=None)
def load_outgroup_alleles(seqid, species):

    if species in {'chri', 'epir'}:
        variants_fn = (
            '/Users/Utente/Documents/UROP2018/phase1.AR3/agc/%s_fake_cnvrt_sort.vcf.gz.vcfnp_cache/variants.%s.npy'
            % (species, seqid)
        )
        variants = np.load(variants_fn)
        pos = variants['POS']
        ref = variants['REF']
        alt = variants['ALT']
        
        outgroup_allele = np.where(alt == b'.', ref, alt)
        
        # fill out to length of chromosome
        gens = phase2_ar1.genome_agamp3
        out = np.empty(len(gens[seqid]), dtype='S1')
        out[:] = b'N'
        out[pos - 1] = outgroup_allele
        
        # Emma edit - creating multiple arrays to make data homogenous 
        pop_alt = np.zeros((len(gens[seqid]),2), dtype = 'U')
        pop_ac = np.zeros((len(gens[seqid]),2), dtype = int)
        pop_ac[:,0]=1
        
        pop_alt[:,0]=out
        
        return pop_ac, pop_alt.astype('U')
        
    else:

        callset_fn = (
            '/Users/Utente/Documents/UROP2018/phase1.AR3/agc/UnifiedGenotyper/%s_ref_ug_vqsr_cnvrt_sort.h5'
            % species
        )
        callset = h5py.File(callset_fn, mode='r')
        
        # compute allele counts
        g = allel.GenotypeDaskArray(callset[seqid]['calldata/genotype'])
        ac = g.count_alleles(max_allele=1).compute()
        an = ac.sum(axis=1).max()

        # obtain helper arrays
        variants = callset[seqid]['variants']
        pos = variants['POS'][:]
        ref = variants['REF'][:]
        alt = variants['ALT'][:]
        
        # don't call allele unless it's fixed
        outgroup_allele = np.empty_like(ref)
        outgroup_allele[:] = b'N'
        loc_ref = ac[:, 0] == an
        loc_alt = ac[:, 1] == an
        outgroup_allele[loc_ref] = ref[loc_ref]
        outgroup_allele[loc_alt] = alt[loc_alt]
        
        # fill out to length of chromosome
        gens = phase2_ar1.genome_agamp3
        out = np.empty(len(gens[seqid]), dtype='S1')
        out[:] = b'N'
        out[pos - 1] = outgroup_allele
        
        # Emma edit - creating multiple arrays to make data homogenous 
        pop_alt = np.empty((len(gens[seqid]),2), dtype='S1')
        pop_ac = np.empty((len(gens[seqid]),2), dtype= int)

        pop_alt[:] = b'N'
        pop_ac[:] = 0

        pop_alt[pos-1,0] = ref
        pop_alt[pos-1,1] = alt
        pop_ac[pos-1,:] = ac

        #return out.astype('U'), ac_all, ref_all.astype('U'), alt_all.astype('U')
        return pop_ac, pop_alt.astype('U')

* EDIT THIS - NOT EDITED OUT NOW
The 'Emma Edit' is an addition to Alistair's original function, that puts the data in the format shown below, and that is compatible to the other functions given below. To extract the various sequences from some of the outgroups use the code above that has been commented out. The function was changed from Alistair's array, that outputted a single output sequence per outgroup, to one that outputs two arrays, the population counts and the allele base array. Making the output data format equal to the population arrays allows for more generic Basic Stat divergence and pi functions to be created.
<br> These output arrays:

- __pop_alt__ = Size equals length of genome, with two columns. At each site gives counts of the number of sequences with the reference base (first column), and number of sequences with the alternate base (second column). 
- __pop_ac__ = Size equals length of genome, with two columns. At each site gives reference base if one is available (first column), and alternate base if it exists(second column). 

Load the data as shown in the example below, for the various outgroups.

In [44]:
seqid = '3L'
#to extract a single sequence
a_ac  , a_alt   = load_outgroup_alleles(seqid, 'arab')
q_ac  , q_alt   = load_outgroup_alleles(seqid, 'quad')
mer_ac, mer_alt = load_outgroup_alleles(seqid, 'meru')
mel_ac, mel_alt = load_outgroup_alleles(seqid, 'mela')
c_ac  , c_alt   = load_outgroup_alleles(seqid, 'chri')
e_ac  , e_alt   = load_outgroup_alleles(seqid, 'epir')

### 3 Site Annotations to Extract

Some data pertaining to the bases is useful, and used to generate the output data. Three key ones are shown below.

__seq_cls__
<br>Differentiates between the various sequence classes below. UTR refers to the untranslated regions on the ends of genes. CDS refers to gene coding sequences. UPSTREAM and DOWNSTREAM refer to the sequences on the ends of and between genes. 
<br>CLS_UPSTREAM = 1
<br>CLS_DOWNSTREAM = 2
<br>CLS_5UTR = 3
<br>CLS_3UTR = 4
<br>CLS_CDS_FIRST = 5
<br>CLS_CDS_MID = 6
<br>CLS_CDS_LAST = 7
<br>CLS_INTRON_FIRST = 8
<br>CLS_INTRON_MID = 9
<br>CLS_INTRON_LAST = 10
<br>Refer to Alistair's build-sequence-classes.ipynb for greater detail.

__codon_position__ 
<br>Denotes the codon position (0,1,2) for every exon base location. (-1) otherwise. 
<br>Refer to Alistair's build-codon-degeneracy.ipynb for greater detail.

__codon_degeneracy__
<br>Denotes the degeneracy of each codon base. (1) denotes a zero-fold site, (4) denotes a four-fold site. 
<br>Refer to Alistair's build-codon-degeneracy.ipynb for greater detail.


In [48]:
seq_cls = zarr.open_group(str(output_dir / 'seq_cls.zarr.zip'))[seqid][:]
codon_position = zarr.open_group(str(output_dir / 'codon_position.zarr.zip'))[seqid][:]
codon_degeneracy = zarr.open_group(str(output_dir / 'codon_degeneracy.zarr.zip'))[seqid][:]


* EDIT - MOVE THE CODE BELOW TO THE BUILD POPULATION SECTION

In [ ]:

callset = phase2_ar1.callset
alt = phase2_ar1.callset[seqid]['variants']['ALT'][:]
pos = phase2_ar1.callset[seqid]['variants']['POS'][:]
allele_counts = phase2_ar1.allele_counts

### 4 Filters Used

#### 4.1 Genome accessibility: 
An important filter that will be used is genome accessibility. Below are relevant exerpts from _Genetic diversity of the African malaria vector Anopheles gambiae_ Supplementary Information 3.4 Genome Accessibility 

The following metrics were computed from alignments for the 765 individual specimens that passed all sample QC criteria defined above: 

 - __No Coverage__: The percentage of individuals with zero coverage at the given position. 
 - __Low Coverage__: The percentage of individuals with low coverage at the given position (less than half modal coverage for the whole chromosome arm).
 - __High Coverage__: The percentage of individuals with high coverage at the given position (more than twice modal coverage for the whole chromosome). 
 - __Ambiguous Alignment__: The percentage of individuals with more than 10% of reads ambiguously aligned (mapping quality zero) at the given position. 
 - __Low Mapping Quality__: The percentage of individuals with average mapping quality less than 30 at the given position. 
 - __Low Read Pairing__: The percentage of individuals with less than 90% of reads aligned as part of a proper pair (mate pair is aligned in expected orientation within a reasonable distance) at the given position.

In [4]:
seqid = '3L'
list(phase2_ar1.accessibility[seqid])

['alt',
 'chrom',
 'coverage',
 'coverage_mq0',
 'filter_dust',
 'filter_high_coverage',
 'filter_high_mq0',
 'filter_low_coverage',
 'filter_low_mq',
 'filter_n',
 'filter_no_coverage',
 'high_coverage',
 'high_mq0',
 'is_accessible',
 'low_coverage',
 'low_mq',
 'low_pairing',
 'no_coverage',
 'pos',
 'ref',
 'ref_masked',
 'ref_n',
 'repeat_dust',
 'repeat_repeatmask',
 'repeat_trf']

At each location the numbers below shows the individuals for which there is no coverage. The total number is 1142 as there are 571 individuals across all 16 populations. The same format is available for all the accessibility-related arrays that pertains to the chromosome sites.

In [ ]:
phase2_ar1.accessibility[seqid]['no_coverage'][:]

The most important one from the list above is __is_accessible__. The final decision was to classify a reference genome position as accessible if and only if all of the following conditions were met: 
 - __Not repeat masked by DUST__ 
 - __No Coverage <= 0.2%__ (at most 1 individual had zero coverage) 
 - __Ambiguous Alignment <= 0.2%__ (at most 1 individual had ambiguous alignments) 
 - __High Coverage <= 2%__ (15 individuals) 
 (to resolve sites where total coverage across all individuals might appear normal but some individuals have excessively low coverage while others have excessively high coverage.)
 - __Low Coverage <= 10%__ (76 individuals)
 - __Low Mapping Quality <= 10%__ (76 individuals) 
 
 <br> For more detailed information or explanations refer to the Supplementary Information of the 1000 genome paper.

In [ ]:
acs = phase2_ar1.accessibility[seqid]['is_accessible'][:]

#### 4.2 Data Filtering (Filter 2.0 as called in the data files)

A whole range of filters have been applied to the data in time. The final filter to be applied has data meet the following conditions: 

<br> For (BFgam, BFcol, KE) the following must be true for all three populations: 
 - The sites must be accessible. 
 - The sites must be biallelic within each population.
 - If the site is polymorphic, it must have the total number of individuals be accounted for in each population.
 
<br> For each outgroup (arab, quad, meru, mela, chri, epir) the following must be true for each outgroup:
 - There must be at least one valid outgroup sequence at each site. 
 
<br>Additionally, filter for four-fold, zero-fold, total filtered individuals.

The function below builds the filter, and the line below it runs it.
<br> The data to be loaded before building the function:
 - pop_ac for (BFgam, BFcol, KE) and (arab, quad, meru, mela, chri, epir).
 - codon_degeneracy, as shown in section 3.

In [ ]:
def new_filter_tot():
    #biallelic for all populations
    BFg_bia = (BFg_ac[:,2]<=0)
    BFc_bia = (BFc_ac[:,2]<=0)
    KE_bia = (KE_ac[:,2]<=0)
    bia_tot = BFg_bia & BFc_bia & KE_bia
    
    #All individuals present
    BFg_full = (BFg_b[:,0]==t4)
    BFc_full = (BFc_b[:,0]==t5)
    KE_full = (KE_b[:,0]==t14)
    full_tot = BFg_full & BFc_full & KE_full
                
    #Valid Outgroup Sites
    sum_a = np.sum(a_ac, axis=1)>= 1
    filter_a = sum_a & np.invert((a_ac[:,0]>0) & (a_alt[:,0]=='N'))
    sum_q = np.sum(q_ac, axis=1)>= 1
    filter_q = sum_q & np.invert((q_ac[:,0]>0) & (q_alt[:,0] =='N'))
    sum_mer = np.sum(mer_ac, axis=1)>= 1
    filter_mer = sum_mer & np.invert((mer_ac[:,0]>0) & (mer_alt[:,0] =='N'))                           
    sum_mel = np.sum(mel_ac, axis=1)>= 1
    filter_mel = sum_mel & np.invert((mel_ac[:,0]>0) & (mel_alt[:,0] =='N'))
    filter_c = (c_alt[:,0]!='N')
    filter_e = (e_alt[:,0]!='N')
    outgroup_full = filter_a & filter_q & filter_mer & filter_mel & filter_c & filter_e
    
    #Accessibility 
    acs = phase2_ar1.accessibility[seqid]['is_accessible'][:]
    
    #Total Filter
    filter_new_tot = acs & bia_tot & full_tot & outgroup_full
    
    #codon degeneracy - 0-fold and 4-fold filters
    fourfold = codon_degeneracy==4
    zerofold = codon_degeneracy==1
    filter_new_0 = filter_new_tot & zerofold 
    filter_new_4 = filter_new_tot & fourfold
    
    return filter_new_tot, filter_new_0, filter_new_4

new_filter_tot, new_filter_0, new_filter_4 = new_filter_tot()

I recommend building the filter once, then saving it as a zarr file and accessing it as such in separate jupyter documents. This makes for much easier access, and means the filter needs to only be built once (it is a very computationally heavy function. The code below shows how to save a simple zarr array, and access it. 

In [ ]:
#creating the zarr arrays from numpy arrays new_filter_tot, new_filter_0, new_filter_4

n_fil_tot = zarr.array(new_filter_tot)
n_fil_0 = zarr.array(new_filter_0)
n_fil_4 = zarr.array(new_filter_4)

#saving the zarr arrays to the output path 

zarr.save(os.path.join(output_dir, 'n_fil_tot_3R.zarr'), n_fil_tot)
zarr.save(os.path.join(output_dir, 'n_fil_0_3R.zarr'), n_fil_0)
zarr.save(os.path.join(output_dir, 'n_fil_4_3R.zarr'), n_fil_4)

#Accessing the filters from a different file/ a different time

new_filter_tot = zarr.load(os.path.join(output_dir, 'n_fil_tot_3L.zarr') )
new_filter_0 =  zarr.load(os.path.join(output_dir, 'n_fil_0_3L.zarr')   )
new_filter_4 =  zarr.load(os.path.join(output_dir, 'n_fil_4_3L.zarr')   )


### 5 Building Population Matrices for (BFgam, BFcol, KE)

pop_ac and pop_alt are built for populations BFgam, BFcol, KE in the same format as they are with outgroup data in section 2. The only difference is that there is no more concept of a 'reference' and 'alternate', but the columns are ordered according to the frequency of the allele. There are also four columns, due the data allowing for more than two alternate alleles.

- __pop_alt__ = Size equals length of genome, with four columns. At each site gives counts of the number of sequences with the most common allele (first column) to least common allele (fourth column), if these alternate alleles exist. 
- __pop_ac__ = Size equals length of genome, with four columns. At each site gives base of most common allele (first column) to least common allele (fourth column), if these alleles exist. 

The columns bases and counts map each other.  

The following functions are meant to be run all together.

In the first cell the genome is loaded and re-shaped. 'a', 'c', 'g', 't' are changed to 'A','C','G','T'. The SNP data is put in a suitable format for the next cell, combining reference and alternate alleles.
<br> In the second cell the counts are loaded. They are paired with the allele data, and reshuffled in order of frequency, with highest frequency alleles going in the first column, second highest in the second column and so on. Alleles that occur zero times are deleted. Intermediary matrices include:
 - __altsa__ = (matrix) contains the bases individuals of that population have at that location in the chromosome, ordered by their frequency. Only contain SNP data.
 - __aca__ = (matrix) contains the numbers of individuals that have the base in the corresponding position in altsa. Only contain SNP data.
 - __b__ = (array) the total number of individuals accounted for in that population at that location. Only contain SNP data.
 - __t__ = (integer) total number of individuals of that population.

In the third cell populations are built by calling on the function just described __build_mc__. All fifteen populations are shown, but only (BFgam, BFcol, KE) need be built.
<br> In the fourth cell the SNP data is re-integrated with the reference sequence to build the final population __pop_ac__ and __pop_alt__. 
<br> In the third cell populations are built by calling on the function just described __build_extended_mc__.

In [ ]:
genome = phase2_ar1.genome_agamp3[seqid][:]
genome_a = np.zeros((len(genome),1), dtype = '|S1')
for i in range (0,len(genome)):
    genome_a[i]=genome[i]
genome = genome_a

#turning lowercase into uppercase
ref = genome[:,0].astype('U')
ref[ref=='a']='A'
ref[ref=='c']='C'
ref[ref=='g']='G'
ref[ref=='t']='T'
ref[ref=='n']='N'


#bringing together all the alternates
refv = ref[pos-1]
refv = np.reshape(refv, (-1,1))
alts_mat = np.concatenate((refv,alt),axis=1)

In [ ]:
def build_mc(pop):
    
    alts = deepcopy(alts_mat)
    
    ac = allel.AlleleCountsArray(allele_counts[seqid][pop])
    totinds = np.max(ac[:,0])
    #print(totinds)


    #making all values that don't map to an alternate base -1
    ac1 = ac.astype(int) 
    ac1[alts=='']=-1

    #resort!
    indices = np.flip(ac1.argsort(),axis = 1)
    row = np.reshape(np.array( np.linspace(0,len(ac)-1,len(ac)), dtype = int),(-1,1))
    table = np.concatenate((row,row,row,row),axis=1)
    atuple = (table,indices)
    altsa = alts[atuple]
    aca = ac1[atuple]

    #delete alleles that occur zero times
    altsa[aca==0] = ''
    aca[aca==0]=-1

    b = deepcopy(aca)
    b[aca==-1]=0
    b = np.sum(b,axis= 1)
    
    return altsa, aca, b, totinds

In [ ]:
#seem to be missing GNcol, which on map has 1 female individual but here seems to have 4. either map mistake, different data, or couldn't identify three of the individuals by gender? seems improbable...
p1a, p1b, p1c, t1 = build_mc('GM')
p2a, p2b, p2c, t2 = build_mc('GW')
p3a, p3b, p3c, t3 = build_mc('GNgam')
p4a, p4b, p4c, t4 = build_mc('BFgam')
p5a, p5b, p5c, t5 = build_mc('BFcol')
p6a, p6b, p6c, t6 = build_mc('CIcol')
p7a, p7b, p7c, t7 = build_mc('GHgam')
p8a, p8b, p8c, t8 = build_mc('GHcol')
p9a, p9b, p9c, t9 = build_mc('GQgam')
p10a, p10b, p10c, t10 = build_mc('GAgam')
p11a, p11b, p11c, t11 = build_mc('CMgam')
p12a, p12b, p12c, t12 = build_mc('AOcol')
p13a, p13b, p13c, t13 = build_mc('UGgam')
p14a, p14b, p14c, t14 = build_mc('KE')
p15a, p15b, p15c, t15 = build_mc('FRgam')


In [ ]:
def build_extended_mc(altsa, aca, b, totinds):
    
    ext_ac = np.ones((len(genome),4), dtype =int)*totinds
    ext_ac[pos-1, 0] = np.reshape (aca[:,0], (-1,1))
    ext_ac[pos-1, 1] = np.reshape (aca[:,1], (-1,1))
    ext_ac[pos-1, 2] = np.reshape (aca[:,2], (-1,1))
    ext_ac[pos-1, 3] = np.reshape (aca[:,3], (-1,1))
    
    ext_alt = np.ones((len(genome),4), dtype =int)*totinds
    ext_alt[pos-1, 0] = np.reshape (altsa[:,0], (-1,1))
    ext_alt[pos-1, 1] = np.reshape (altsa[:,1], (-1,1))
    ext_alt[pos-1, 2] = np.reshape (altsa[:,2], (-1,1))
    ext_alt[pos-1, 3] = np.reshape (altsa[:,3], (-1,1))
    
    ext_b = np.ones((len(genome),1), dtype = int)*(totinds) 
    ext_b[pos-1] = np.reshape(b,(-1,1))

    #pos is shared between every population for a given chromosome
    pos1 = np.linspace(1,len(genome),len(genome), dtype = int)
    pos1 = np.reshape (pos1, (-1,1))
    
    return ext_ac, ext_alt, ext_b, pos1

In [ ]:
BFg_ac, BFg_alt, BFg_b = build_extended_mc(p4a, p4b, p4c, t4)
BFc_ac, BFc_alt, BFc_b = build_extended_mc(p5a, p5b, p5c, t5)
KE_ac, KE_alt, KE_b = build_extended_mc(p14a, p14b, p14c, t14)

### 6 The 'Basic Stats'

#### 6.1 Calculating Pi within populations

The matrix __pop_ac__ , an output from sections 2 and 5 , is inputted into the function below to calculate pi within each population.

In [ ]:
def pi_cal(pop_ac):
    pop_ac[pop_ac==(-1)]=0
    pi_array = allel.mean_pairwise_difference(pop_ac)
    return pi_array

The pi array is built for a population, and values are found by finding the mean of the array as filtered by either __new_filter_tot__, __new_filter_0__ and __new_filter_4__ as shown in section 4.

In [ ]:
BFg_pi = pi_cal(BFg_ac)
np.mean(BFg_pi[new_filter_4])

#### 6.2 Divergence Calculations

Formatting all outgroup and population data into __pop_ac__ and __pop_alt__ has the advantage of being able to use very generalizable functions. The one below requires inputs __pop_ac__ and __pop_alt__ from any two outgroup or Gambiae populations (populations a and b) to give an array of divergences __div__. To reduce the amount of values stored, this is collapsed within the function to output __div1__, which gives the overall diverges values of sites filtered with __new_filter_tot__, __new_filter_0__ and __new_filter_4__.

* ADD BETTER DESCRIPTION OF COLLAPSE * 

The divergence calculation is given below and inplemented in __divving__.

divergence = 1 - [(f'A', p1) * (f 'A' ,p2)  + (f'C', p1) * (f 'C' ,p2)  + (f'G', p1) * (f 'G' ,p2)]
<br> f'A', p1  refers to the frequency of base A at that specific site in population 1. p1 and p2 are populations 1 and 2. 
<br> f'A', f'C', f'G', f'T' are the frequencies of bases A,C,G,T respectively.



In [ ]:
def divving(pop_a_ac, pop_a_alt, pop_b_ac, pop_b_alt):
    
    pop_a_ac[pop_a_ac==-1]=0
    pop_b_ac[pop_b_ac==-1]=0
    
    div = np.zeros((len(genome),1), dtype = float)
    pop_b_ac1 = pop_b_ac/np.reshape(np.sum(pop_b_ac, axis = 1), (-1,1))
    pop_a_ac1 = pop_a_ac/np.reshape(np.sum(pop_a_ac, axis = 1), (-1,1))
    
    div[pop_b_alt[:,0]==pop_a_alt[:,0]]=np.reshape(pop_b_ac1[pop_b_alt[:,0]==pop_a_alt[:,0],0], (-1,1)) *np.reshape((pop_a_ac1[pop_b_alt[:,0]==pop_a_alt[:,0],0]),(-1,1))
    div[pop_b_alt[:,0]==pop_a_alt[:,1]]=np.reshape(pop_b_ac1[pop_b_alt[:,0]==pop_a_alt[:,1],0], (-1,1)) *np.reshape((pop_a_ac1[pop_b_alt[:,0]==pop_a_alt[:,1],1]),(-1,1))
    div[pop_b_alt[:,1]==pop_a_alt[:,0]]= div[pop_b_alt[:,1]==pop_a_alt[:,0]] + np.reshape(pop_b_ac1[pop_b_alt[:,1]==pop_a_alt[:,0],1], (-1,1)) *np.reshape((pop_a_ac1[pop_b_alt[:,1]==pop_a_alt[:,0],0]),(-1,1))
    div[pop_b_alt[:,1]==pop_a_alt[:,1]]= div[pop_b_alt[:,1]==pop_a_alt[:,1]] + np.reshape(pop_b_ac1[pop_b_alt[:,1]==pop_a_alt[:,1],1], (-1,1)) *np.reshape((pop_a_ac1[pop_b_alt[:,1]==pop_a_alt[:,1],1]),(-1,1))
    
    print('doing...')
    div = 1 - div
    div1 = [np.mean(div[new_filter_0]),np.mean(div[new_filter_0]),np.mean(div[new_filter_4])]
    
    #return div
    return div1

The cells below merely display building divergences and displaying them.

In [ ]:
div_BFg_a     =  divving(BFg_ac    , BFg_alt    , a_ac     , a_alt      )
div_BFg_q     =  divving(BFg_ac    , BFg_alt    , q_ac     , q_alt      )
div_BFg_mer   =  divving(BFg_ac    , BFg_alt    , mer_ac   , mer_alt    )
div_BFg_mel   =  divving(BFg_ac    , BFg_alt    , mel_ac   , mel_alt    )
div_BFg_c     =  divving(BFg_ac    , BFg_alt    , c_ac     , c_alt      )
div_BFg_e     =  divving(BFg_ac    , BFg_alt    , e_ac     , e_alt      )

In [ ]:
print(['BFg', 'arab', div_BFg_a   ])
print(['BFg', 'quad', div_BFg_q   ])
print(['BFg', 'meru', div_BFg_mer ])
print(['BFg', 'mela', div_BFg_mel ])
print(['BFg', 'chri', div_BFg_c   ])
print(['BFg', 'epir', div_BFg_e   ])

#### 6.3 Shared Polymorphisms

The inputs to the function below are __pop_ac__ for a population a and population b. 
The outputs are:
 - __shar__: an array that gives a site a value 1 if it is a site with a shared polymorphism, or 0 if it is not. The definiton of a shared polymorphism is that the same two (and only those two) alleles exist in both populations, although they may have different frequencies.
 - __descriptor__: a more expanded array that gives the value 1 if both populations have the same monomorphic site, the value 2 for shared polymorphic sites, the value 3 for mixed polymorphic sites, the value 4 for polymorphic sites that completely differ in the alleles that constitute them (only one of the populations need have a polymorphic site), and value 5 for different monomorphic sites. 

In [ ]:
def shared_poly(pop_a_alt, pop_b_alt):
    
    pop_a_alt = pop_a_alt[:,0:2]
    pop_b_alt = pop_b_alt[:,0:2]
    
    pop_a_alt[pop_a_alt=='N']=''
    pop_a_alt[pop_a_alt=='.']=''
    pop_b_alt[pop_b_alt=='N']=''
    pop_b_alt[pop_b_alt=='.']=''
    
    pop_b_altf = np.flip(pop_b_alt, axis=1)

    shar = np.zeros((len(genome)), dtype = bool)
    shar[((pop_a_alt==pop_b_alt)[:,0]& (pop_a_alt==pop_b_alt)[:,1]) & (pop_a_alt[:,1]!='')] = 1
    shar[((pop_a_alt==pop_b_altf)[:,0]& (pop_a_alt==pop_b_altf)[:,1]) & (pop_a_alt[:,1]!='')] = 1
    
    #shar1 = [np.mean(shar[new_filter_tot]) , np.mean(shar[new_filter_0]) , np.mean(shar[new_filter_4])]
    print('doing...')
    
    descriptor = np.zeros((len(genome)), dtype = int)
    
    #Fixed Similar
    descriptor[((pop_a_alt==pop_b_alt)[:,0]& (pop_a_alt==pop_b_alt)[:,1]) & (pop_a_alt[:,1]=='')] = 1
    # shared polymorphisms
    descriptor[shar == 1] = 2
    #Polymorphic different
    i1 = ((pop_a_alt[:,1]!=pop_b_alt[:,0]) & (pop_a_alt[:,0]!=pop_b_alt[:,1]))
    i2 = (pop_a_alt[:,1]=='')
    descriptor[((pop_a_alt[:,0]!=pop_b_alt[:,0]) & (pop_a_alt[:,0]!=pop_b_alt[:,1])) & np.any([i1,i2], axis =0)] = 3
    #Fixed Different
    descriptor[((pop_a_alt!=pop_b_alt)[:,0]& (pop_a_alt==pop_b_alt)[:,1]) & (pop_a_alt[:,1]=='')] = 4
    
    return shar, descriptor

In [ ]:
print([np.sum([new_filter_0])])
print(['Mono_same', 'Poly_same','Poly_mix', 'Poly_diff', 'Mono_diff'])
print([ np.sum(  (descriptor_a==1)[new_filter_0]), np.sum(  (descriptor_a==2)[new_filter_0]),np.sum(  (descriptor_a==0)[new_filter_0]),np.sum(  (descriptor_a==3)[new_filter_0]),np.sum(  (descriptor_a==4)[new_filter_0])])
print([ np.sum(  (descriptor_q==1)[new_filter_0]), np.sum(  (descriptor_q==2)[new_filter_0]),np.sum(  (descriptor_q==0)[new_filter_0]),np.sum(  (descriptor_q==3)[new_filter_0]),np.sum(  (descriptor_q==4)[new_filter_0])])
print([ np.sum((descriptor_mer==1)[new_filter_0]), np.sum((descriptor_mer==2)[new_filter_0]),np.sum((descriptor_mer==0)[new_filter_0]),np.sum((descriptor_mer==3)[new_filter_0]),np.sum((descriptor_mer==4)[new_filter_0])])
print([ np.sum((descriptor_mel==1)[new_filter_0]), np.sum((descriptor_mel==2)[new_filter_0]),np.sum((descriptor_mel==0)[new_filter_0]),np.sum((descriptor_mel==3)[new_filter_0]),np.sum((descriptor_mel==4)[new_filter_0])])
print([ np.sum(  (descriptor_c==1)[new_filter_0]), np.sum(  (descriptor_c==2)[new_filter_0]),np.sum(  (descriptor_c==0)[new_filter_0]),np.sum(  (descriptor_c==3)[new_filter_0]),np.sum(  (descriptor_c==4)[new_filter_0])])
print([ np.sum(  (descriptor_e==1)[new_filter_0]), np.sum(  (descriptor_e==2)[new_filter_0]),np.sum(  (descriptor_e==0)[new_filter_0]),np.sum(  (descriptor_e==3)[new_filter_0]),np.sum(  (descriptor_e==4)[new_filter_0])])


### 7 Creating the DAFs

Note: Function below has not been updated to use the new standardized data formats, and therefore would have to brushed up to be used in conjunction with current functions. __out1__ and __out2__ need to be changed to suit __pop_ac__ and __pop_alt__. 

__Method Used:__
(A.)  Calculating an ancestral sequence

If there is a match in the population with out1, write match to anc, else
<br>&nbsp;&nbsp;&nbsp;&nbsp;If there is a match with out2, write match to anc, else
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;If there s a match between out1 and out2, take as anc, else
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;If no match, write ND1

(B.) Counting the derived alleles

If the ancestral has ND1, write ND2, else
<br>&nbsp;&nbsp;&nbsp;&nbsp;If the population has only one non-match to a valid ancestral base, count the non-match as derived, else
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;If the population has more than one non-matches, write ND3


In [ ]:
#building the ancestral 
def build_ancestral(out1,out2):
    anc = np.zeros((len(genome)),dtype = '|S1').astype('U')

    anc[out1==REF]=out1[out1==REF]

    anc[(ALT_1!='') & (out1==ALT_1)]=out1[(ALT_1!='') & (out1==ALT_1)]
    anc[(ALT_2!='') & (out1==ALT_2)]=out1[(ALT_2!='') & (out1==ALT_2)]
    anc[(ALT_3!='') & (out1==ALT_3)]=out1[(ALT_3!='') & (out1==ALT_3)]
    anc[(anc=='') & (REF!='') & (out2==REF)]= out2[(anc=='') & (REF!='') & (out2==REF)]
    anc[(anc=='') & (ALT_1!='') & (out2==ALT_1)]=out2[(anc=='') & (ALT_1!='') & (out2==ALT_1)]
    anc[(anc=='') & (ALT_2!='') & (out2==ALT_2)]=out2[(anc=='') & (ALT_2!='') & (out2==ALT_2)]
    anc[(anc=='') & (ALT_3!='') & (out2==ALT_3)]=out2[(anc=='') & (ALT_3!='') & (out2==ALT_3)]
    anc[(anc=='') & (out1==out2)]=out1[(anc=='') & (out1==out2)]
    anc[anc=='']='M'

    return anc


In [ ]:
def build_derived(anc):
    der = np.zeros((len(genome),1),dtype = int).astype('U')
    
    der[REF!=anc]= ext_ac_0[REF!=anc]
    der[(ALT_1!='') & (ALT_1!=anc)] = ext_ac_1[(ALT_1!='') & (ALT_1!=anc)]
    #unnecessary if only choosing bi-allelic sites. 
    #der[(ALT_2!='') & (ALT_2!=anc)] = der[(ALT_2!='') & (ALT_2!=anc)] + ext_ac_2[(ALT_2!='') & (ALT_2!=anc)]
    #der[(ALT_3!='') & (ALT_3!=anc)] = der[(ALT_3!='') & (ALT_3!=anc)] + ext_ac_3[(ALT_3!='') & (ALT_3!=anc)]

    der[(ALT_2!='')] = -2
    der[(REF!=anc) & (ALT_1!='') & (ALT_1!=anc)] = -2

    der[anc=='M']=-1
    # M = ND1 -1= ND2 -2= ND3

    return der

### 8 Creating the MAFs

Creating the MAFs only require accessing __pop_ac__ of a given population. 
<br>Allele counts were organized above by frequency at each site, so the second column, of the second highest frequency allele counts, forms the entirely of the MAFs. 
<br>An example MAF of four-fold sites of BFgam is given below.
<br> The values are rerouted through the __create_MAFs__ function.

In [ ]:
#t4 is equal to the total individuals of population 4, BFgam
def create_MAFs(unique,counts, totinds):
    counts_new = np.zeros(int((totinds/2)+1), dtype = int)
    for i in range(0,len(unique)):
        al = unique[i]
        counts_new[al] = counts[i]       
    return counts_new

unique, counts = np.unique(BFg_ac[:,1][new_filter_4], return_counts=True)
counts_new = create_MAFs(unique, counts, t4))
list(counts_new)

### 9 Building the Conserved, Non-conserved Sites

Conserved sites are defined as sites where the amino acid coded for by chri is the same as that coded for by epir. 

This therefore requires determining what amino acids are coded for by each species at each site. The following functions deal with that (and are an adaptation of Alistair's codon degeneracy code). 

In this first cell, data tables and features are imported. 

In [ ]:
tbl_features = etl.frompickle(os.path.join(output_dir, 'tbl_features.pkl.gz'))
from Bio.Data import CodonTable
standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
bases = ['A', 'C', 'T', 'G']
codon_table = standard_table.forward_table.copy()
for codon in standard_table.stop_codons:
    codon_table[codon] = '*'
from Bio.Seq import Seq

In this cell, appropriate formats for the reference data of chri and epir are defined with __c_ref_aa__ and __e_ref_aa__. These will contain the amino acids coded for at each site.

__chri_ref__ is created, which contains the outgroup nucleotide sequence. c_2L, c_2R,... c_X are loaded previously as shown in section 2. __epir_ref__ is similarly created. 

In [ ]:
c_ref_aa = {k: np.zeros(len(genome[k]), dtype='U')
                  for k in genome.keys()}
e_ref_aa = {k: np.zeros(len(genome[k]), dtype='U')
                  for k in genome.keys()}
#load these separately 
chri_ref = {k: np.zeros(len(genome[k]), dtype='U')
                  for k in genome.keys()}

chri_ref['2L']  =  np.reshape( c_2L[:,0], len(e_2L)) 
chri_ref['2R']  =  np.reshape( c_2R[:,0], len(e_2R)) 
chri_ref['3L']  =  np.reshape( c_3L[:,0], len(e_3L)) 
chri_ref['3R']  =  np.reshape( c_3R[:,0], len(e_3R)) 
chri_ref['X']   =  np.reshape( c_X[:,0], len(e_X)) 
chri_ref['Y_unplaced'][:]   =  'N'
chri_ref['UNKN'][:]   =  'N'


The code below builds __c_ref_aa__ by using the codon position and phase data for the coding section sequences, and consulting the codon table. To build __e_ref_aa__ change __chri_ref__ to __epir_ref__ (built equally as above), and __c_ref_aa__ to __e_ref_aa__. 

In [ ]:
for cds in tbl_features.eq('type', 'CDS').true('is_canonical').records():
    if cds.seqid not in genome:
        continue
    if cds.strand == '+':
        seq = ''.join(chri_ref[cds.seqid][cds.start:cds.stop])
        phase = int(cds.phase)
        for i in range(cds.length):
            codon_pos = (i - phase) % 3
            if codon_pos == 0:
                codon_start = i
                codon_stop = i+3
            elif codon_pos == 1:
                codon_start = i-1
                codon_stop = i+2
            elif codon_pos == 2:
                codon_start = i-2
                codon_stop = i+1
            else:
                raise RuntimeError('unexpected codon_pos', codon_pos)
            if codon_start < 0:
                continue
            if codon_stop > cds.length:
                continue
            codon = seq[codon_start:codon_stop]
            if 'N' not in codon:
                ref_aa = codon_table[codon]
                c_ref_aa[cds.seqid][cds.start + i] = ref_aa

    else:
        seq = ''.join(chri_ref[cds.seqid][cds.start:cds.stop]).upper()
        seq = str(Seq(seq).reverse_complement())
        phase = int(cds.phase)
        for i in range(cds.length):
            codon_pos = (i - phase) % 3
            if codon_pos == 0:
                codon_start = i
                codon_stop = i+3
            elif codon_pos == 1:
                codon_start = i-1
                codon_stop = i+2
            elif codon_pos == 2:
                codon_start = i-2
                codon_stop = i+1
            else:
                raise RuntimeError('unexpected codon_pos', codon_pos)
            if codon_start < 0:
                continue
            if codon_stop > cds.length:
                continue
            codon = seq[codon_start:codon_stop]
            if 'N' not in codon:
                ref_aa = codon_table[codon]
                alt_aa = [a for a in aas if a != ref_aa]
                nonsyn = len(alt_aa)
                seq_index = cds.stop - i - 1
                c_ref_aa[cds.seqid][seq_index] = ref_aa
                
#                 print(i, codon_pos, codon, codons, ref_aa, aas, deg, nonsyn, seq_index, genome[cds.seqid][seq_index])

The code below builds __con_all__ and __ncon_all__, the arrays that define the filters for all conserved and non-conserved sites in the coding regions of the sequences. It compares __c_ref_aa__ to __e_ref_aa__. Sites that do not code for an amino acid are not included in either __con_all__ or __ncon_all__. 

In [ ]:
#compare c_ref_aa with e_ref_aa
con_all = {k: np.zeros(len(genome[k]), dtype= int)
                  for k in genome.keys()}
ncon_all = {k: np.zeros(len(genome[k]), dtype= int)
                  for k in genome.keys()}

c1_2L = c_ref_aa['2L'][:]
c1_2R = c_ref_aa['2R'][:]
c1_3L = c_ref_aa['3L'][:]
c1_3R = c_ref_aa['3R'][:]
c1_X  = c_ref_aa['X'][:]

e1_2L = e_ref_aa['2L'][:]
e1_2R = e_ref_aa['2R'][:]
e1_3L = e_ref_aa['3L'][:]
e1_3R = e_ref_aa['3R'][:]
e1_X  = e_ref_aa['X'][:]

con_all['2L'] = (c1_2L!='') & (e1_2L!='') & (c1_2L == e1_2L)
con_all['2R'] = (c1_2R!='') & (e1_2R!='') & (c1_2R == e1_2R)
con_all['3L'] = (c1_3L!='') & (e1_3L!='') & (c1_3L == e1_3L)
con_all['3R'] = (c1_3R!='') & (e1_3R!='') & (c1_3R == e1_3R)
con_all['X']  = (c1_X!='')  & (e1_X!='')  & (c1_X == e1_X)

ncon_all['2L'] = (c1_2L!='') & (e1_2L!='') & (c1_2L != e1_2L)
ncon_all['2R'] = (c1_2R!='') & (e1_2R!='') & (c1_2R != e1_2R)
ncon_all['3L'] = (c1_3L!='') & (e1_3L!='') & (c1_3L != e1_3L)
ncon_all['3R'] = (c1_3R!='') & (e1_3R!='') & (c1_3R != e1_3R)
ncon_all['X']  = (c1_X!='')  & (e1_X!='')  & (c1_X  != e1_X)

As with the filters, it is suggested to run this once, save as a zarr group, and access it as such in other files/ other moments.

In [ ]:
#saving as a zarr group
zarr.save_group(os.path.join(output_dir, 'chri_codons.zarr.zip'), 
                **c_ref_all)
zarr.save_group(os.path.join(output_dir, 'epir_codons.zarr.zip'), 
                **e_ref_all)

#Accessing zarr group
con_all = zarr.open_group(str(output_dir / 'con_all.zarr.zip'))
ncon_all = zarr.open_group(str(output_dir / 'ncon_all.zarr.zip'))
con = con_all[seqid][:]
ncon = ncon_all[seqid][:]

To filter by PEST REFERENCE amino acids, repeat cells (1,2,3) with the genome reference data rather than __chri_ref__ or __epir_ref__. 

To find the degeneracies of chri and epir, insert the sections below in cell 3 of this section as appropriate. Also save and access as a zarr group.

In [ ]:
#extracting degeneracies modificiation!
#insert section below before the first 'ref_aa =...'
                codons = None
                if codon_pos == 0:
                    codons = [b + codon[1:] for b in bases]
                elif codon_pos == 1:
                    codons = [codon[0] + b + codon[2] for b in bases]
                elif codon_pos == 2:
                    codons = [codon[:2] + b for b in bases]
                aas = [codon_table[c] for c in codons]
                # assign degeneracy
                aas_ct = collections.Counter(aas)
                if len(aas_ct) == 1:
                    # 4-fold degenerate
                    deg = 4
                elif len(aas_ct) == 4:
                    # 0-fold degenerate
                    deg = 1
                elif len(aas_ct) == 2 and tuple(aas_ct.values()) == (2, 2):
                    # simple 2-fold degeneracy
                    deg = 2
                else:
                    # complex 2-fold degenerate
                    deg = 3
                # assign number of nonsyn mutations
                codon_degeneracy[cds.seqid][cds.start + i] = deg
                

#insert section below before the second 'ref_aa =...'
                codons = None
                if codon_pos == 0:
                    codons = [b + codon[1:] for b in bases]
                elif codon_pos == 1:
                    codons = [codon[0] + b + codon[2] for b in bases]
                elif codon_pos == 2:
                    codons = [codon[:2] + b for b in bases]
                aas = [codon_table[c] for c in codons]
                # assign degeneracy
                aas_ct = collections.Counter(aas)
                if len(aas_ct) == 1:
                    # 4-fold degenerate
                    deg = 4
                elif len(aas_ct) == 4:
                    # 0-fold degenerate
                    deg = 1
                elif len(aas_ct) == 2 and tuple(aas_ct.values()) == (2, 2):
                    # simple 2-fold degenerate
                    deg = 2
                else:
                    # complex 2-fold degenerate
                    deg = 3
                # assign number of nonsyn mutations
                codon_degeneracy[cds.seqid][seq_index] = deg              

### 10 Selecting Genes

A filter is created below that filters for the coding sections of relevant genes in a particular chromosome. 
<br> The filter takes as input the __seqid__ of the particular chromosome (eg: '2L'), and the array __selected_genes__ that contains the names of the genes of interest 

In [ ]:
# Example array of relevant genes
selected_genes = ['AGAP004677', 'AGAP004678','AGAP004688','AGAP004698','AGAP004718']

def filter_genes( selected_genes, seqid):
    tbl_features = etl.frompickle(os.path.join(output_dir, 'tbl_features.pkl.gz'))
    
    start_stop = np.zeros((53131,2), dtype = int)
    for mrna in tbl_features.eq('type', 'mRNA').true('is_canonical').records():
        if mrna.parent in selected_genes:
            for cds in tbl_features.eq('type', 'CDS').true('is_canonical').records():
                if cds.parent==mrna.ID:
                    start_stop[i,0] = cds.start
                    start_stop[i,1] = cds.stop
    start_stop = start_stop[0: (np.sum(start_stop[:,0]!=0))]
    
    genome = phase2_ar1.genome_agamp3[seqid][:]
    gene_filter = np.zeros(len(genome), dtype = int)
    for i in range (0,len(ss)):
        start = start_stop[i,0]
        stop = start_stop[i,1]
        gene_filter[start:stop]=1
    
    return gene_filter

* OUTPUT ALL THE CHROMOSOMES INSTEAD * 
* DO MAPPING GENES *


### 11 Visualizing Data

The pandas module can be used to customize tables with which to visualize data.
<br> The table below shows all sites that are filtered for with Filter 2.0. The columns show the alleles of populations (BFg, BFc, KE) and outgroups (arab, quad, meru, mela, chri, epir). 

In [ ]:
def build_df():
    df = pandas.DataFrame.from_items([
        ('POS', np.arange(1,len(genome)+1)[new_filter_tot]),
        ('Deg', codon_degeneracy[new_filter_tot]),
        ('BFg', BFg_alt[:,0][new_filter_tot]),
        #('BFg', BFg_ac [:,0][new_filter_tot]),
        ('BFg', BFg_alt[:,1][new_filter_tot]),
        #('BFg', BFg_ac [:,1][new_filter_tot]),
        ('BFc', BFc_alt[:,0][new_filter_tot]),
        #('BFc', BFc_ac [:,0][new_filter_tot]),
        ('BFc', BFc_alt[:,1][new_filter_tot]),
        #('BFc', BFc_ac [:,1][new_filter_tot]),
        ('KE' , KE_alt [:,0][new_filter_tot]),
        #('KE' , KE_ac  [:,0][new_filter_tot]),
        ('KE' , KE_alt [:,1][new_filter_tot]),
        #('KE' , KE_ac  [:,1][new_filter_tot]),
        ('A', a_alt  [:,0][new_filter_tot]),
        #('A', a_ac   [:,0][new_filter_tot]),
        ('A', a_alt  [:,1][new_filter_tot]),
        #('A', a_ac   [:,1][new_filter_tot]),
        ('Q', q_alt  [:,0][new_filter_tot]),
        #('Q', q_ac   [:,0][new_filter_tot]),
        ('Q', q_alt  [:,1][new_filter_tot]),
        #('Q', q_ac   [:,1][new_filter_tot]),
        ('Mer', mer_alt[:,0][new_filter_tot]),
        #('Mer', mer_ac [:,0][new_filter_tot]),
        ('Mer', mer_alt[:,1][new_filter_tot]),
        #('Mer', mer_ac [:,1][new_filter_tot]),
        ('Mel', mel_alt[:,0][new_filter_tot]),
        #('Mel', mel_ac [:,0][new_filter_tot]),
        ('Mel', mel_alt[:,1][new_filter_tot]),
        #('Mel', mel_ac [:,1][new_filter_tot]),
        ('C', c_alt  [:,0][new_filter_tot]),
        ('E', e_alt  [:,0][new_filter_tot]),
    ])
    
    return df

build_df()